# LRS3 Extract Images from video

## Imports

In [8]:
from pathlib import Path
from tqdm import tqdm
from more_itertools import ilen
from collections import defaultdict
import math
from multiprocessing import Pool

In [12]:
data_dir = Path("/mnt/U/Datasets/lrs3pretrain/lrs3_test_v0.4/test/")
images_dir = Path("/mnt/U/Datasets/lrs3pretrain/test_audio/")

In [13]:
speakers = list(data_dir.glob("*"))
utt_per_spk = {x.name: list(x.glob("*.mp4")) for x in speakers}
n_utt_per_spk = {x: len(y) for x, y in utt_per_spk.items()}
n_utterances = sum(n_utt_per_spk.values())

In [14]:
def get_out_path(spk_utt):
    spk, utt = spk_utt
    out_path = images_dir / spk / (utt.stem + ".wav")
    if not out_path.parent.exists():
        out_path.parent.mkdir(parents=True, exist_ok=True)
    return out_path

def extract_images(utt_outpath):
    utt, out_path = utt_outpath
    !ffmpeg -i {utt} {out_path} -loglevel quiet

In [15]:
spk_utts = []
for spk, utts in utt_per_spk.items():
    spk_utts += [(spk, utt) for utt in utts]
paths = list(map(get_out_path, spk_utts))
spk_paths = list(zip([x[1] for x in spk_utts], paths))

In [16]:
p = Pool(32)
list(tqdm(p.imap_unordered(extract_images, spk_paths), total=n_utterances))

100%|██████████| 1321/1321 [00:26<00:00, 50.62it/s]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,